# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 51 new papers today
          25 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/24 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.07273


extracting tarball to tmp_2302.07273...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.07274


extracting tarball to tmp_2302.07274... done.
Retrieving document from  https://arxiv.org/e-print/2302.07277


...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.07808


extracting tarball to tmp_2302.07808... done.
Retrieving document from  https://arxiv.org/e-print/2302.07853


extracting tarball to tmp_2302.07853...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2302.07853/aa.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'tables/table_peaks.tex' from 'tmp_2302.07853/tables/table_peaks.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'tables/ages_table.tex' from 'tmp_2302.07853/tables/ages_table.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07277-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07277) | **On the nature of disks at high redshift seen by JWST/CEERS with  contrastive learning and cosmological simulations**  |
|| J. Vega-Ferrero, et al. -- incl., <mark>A. Pillepich</mark>, <mark>S. L. Finkelstein</mark> |
|*Appeared on*| *2023-02-16*|
|*Comments*| *25 pages, 23 figures. Submitted to ApJ. Comments welcome*|
|**Abstract**| Visual inspections of the first optical rest-frame images from JWST have indicated a surprisingly high fraction of disk galaxies at high redshifts. Here we alternatively apply self-supervised machine learning to explore the morphological diversity at $z \geq 3$. Our proposed data-driven representation scheme of galaxy morphologies, calibrated on mock images from the TNG50 simulation, is shown to be robust to noise and to correlate well with physical properties of the simulated galaxies, including their 3D structure. We apply the method simultaneously to F200W and F356W galaxy images of a mass-complete sample ($M_*/M_\odot>10^9$) at $z \geq 3$ from the first JWST/NIRCam CEERS data release. We find that the simulated and observed galaxies do not populate the same manifold in the representation space from contrastive learning, partly because the observed galaxies tend to be more compact and more elongated than the simulated galaxies. We also find that about half the galaxies that were visually classified as disks based on their elongated images actually populate a similar region of the representation space than spheroids, which according to the TNG50 simulation is occupied by objects with low stellar specific angular momentum and non-oblate structure. This suggests that the disk fraction at $z > 3$ as evaluated by visual classification may be severely overestimated by misclassifying compact, elongated galaxies as disks. Deeper imaging and/or spectroscopic follow-ups as well as comparisons with other simulations will help to unambiguously determine the true nature of these galaxies. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07497-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07497) | **OGLE-2017-BLG-1038: A Possible Brown-dwarf Binary Revealed by Spitzer  Microlensing Parallax**  |
|| Amber Malpas, et al. -- incl., <mark>Andrew Gould</mark>, <mark>Antonio Herrera Martin</mark>, <mark>Calen B. Henderson</mark>, <mark>Seung-Lee Kim</mark>, <mark>Chung-Uk Lee</mark>, <mark>Dong-Joo Lee</mark>, <mark>Yongseok Lee</mark> |
|*Appeared on*| *2023-02-16*|
|*Comments*| *20 pages, 11 figures, 4 tables*|
|**Abstract**| We report the analysis of microlensing event OGLE-2017-BLG-1038, observed by the Optical Gravitational Lensing Experiment, Korean Microlensing Telescope Network, and Spitzer telescopes. The event is caused by a giant source star in the Galactic Bulge passing over a large resonant binary lens caustic. The availability of space-based data allows the full set of physical parameters to be calculated. However, there exists an eightfold degeneracy in the parallax measurement. The four best solutions correspond to very-low-mass binaries near ($M_1 = 170^{+40}_{-50} M_J$ and $M_2 = 110^{+20}_{-30} M_J$), or well below ($M_1 = 22.5^{+0.7}_{-0.4} M_J$ and $M_2 = 13.3^{+0.4}_{-0.3} M_J$) the boundary between stars and brown dwarfs. A conventional analysis, with scaled uncertainties for Spitzer data, implies a very-low-mass brown dwarf binary lens at a distance of 2 kpc. Compensating for systematic Spitzer errors using a Gaussian process model suggests that a higher mass M-dwarf binary at 6 kpc is equally likely. A Bayesian comparison based on a galactic model favors the larger-mass solutions. We demonstrate how this degeneracy can be resolved within the next ten years through infrared adaptive-optics imaging with a 40 m class telescope. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07502-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07502) | **IC 5146 dark Streamer: is a first reliable candidate of edge collapse,  hub-filament systems, and intertwined sub-filaments?**  |
|| L. K. Dewangan, et al. -- incl., <mark>C. W. Lee</mark> |
|*Appeared on*| *2023-02-16*|
|*Comments*| *22 pages, 11 figures, Accepted for publication in The Astrophysical Journal*|
|**Abstract**| The paper presents an analysis of multi-wavelength data of a nearby star-forming site IC 5146 dark Streamer (d $\sim$600 pc), which has been treated as a single and long filament, fl. Two hub-filament systems (HFSs) are known toward the eastern and the western ends of fl. Earlier published results favor the simultaneous evidence of HFSs and the end-dominated collapse (EDC) in fl. Herschel column density map (resolution $\sim$13$''$.5) reveals two intertwined sub-filaments (i.e., fl-A and fl-B) toward fl, displaying a nearly double helix-like structure. This picture is also supported by the C$^{18}$O(3-2) emission. The scenario "fray and fragment" may explain the origin of intertwined sub-filaments. In the direction of fl, two cloud components around 2 and 4 km s$^{-1}$ are depicted using the $^{13}$CO(1-0) and C$^{18}$O(1-0) emission, and are connected in velocity space. The HFSs are spatially found at the overlapping areas of these cloud components and can be explained by the cloud-cloud collision scenario. Non-thermal gas motion in fl with larger Mach number is found. The magnetic field position angle measured from the filament's long axis shows a linear trend along the filament. This signature is confirmed in the other nearby EDC filaments, presenting a more quantitative confirmation of the EDC scenario. Based on our observational outcomes, we witness multiple processes operational in IC 5146 Streamer. Overall, the Streamer can be recognized as the first reliable candidate of edge collapse, HFSs, and intertwined sub-filaments together. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07700-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07700) | **Self-Supervised Learning for Modeling Gamma-ray Variability in Blazars**  |
|| <mark>Aryeh Brill</mark> |
|*Appeared on*| *2023-02-16*|
|*Comments*| *2nd Annual AAAI Workshop on AI to Accelerate Science and Engineering (AI2ASE), this https URL Updated reference and typo correction in this version (H=32, not 64)*|
|**Abstract**| Blazars are active galactic nuclei with relativistic jets pointed almost directly at Earth. Blazars are characterized by strong, apparently stochastic flux variability at virtually all observed wavelengths and timescales, from minutes to years, the physical origin of which is still poorly understood. In the high-energy gamma-ray band, the Large Area Telescope aboard the Fermi space telescope (Fermi-LAT) has conducted regular monitoring of thousands of blazars since 2008. Deep learning can help uncover structure in gamma-ray blazars' complex variability patterns that traditional methods based on parametric statistical modeling or manual feature engineering may miss. In this work, we propose using a self-supervised Transformer encoder architecture to construct an effective representation of blazar gamma-ray variability. Measurement errors, upper limits, and missing data are accommodated using learned encodings. The model predicts a set of quantiles for the flux probability distribution at each time step, an architecture naturally suited for describing data generated by a stochastic process. As a proof of concept for how the model output can be analyzed to extract scientifically relevant information, a preliminary search for weekly-timescale time-reversal asymmetry in gamma-ray blazar light curves was conducted, finding no significant evidence for asymmetry. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07714-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07714) | **TOI-3984 A b and TOI-5293 A b: two temperate gas giants transiting mid-M  dwarfs in wide binary systems**  |
|| Caleb I. Cañas, et al. -- incl., <mark>Jessica Libby-Roberts</mark>, <mark>Andrea S.J. Lin</mark>, <mark>Suvrath Mahadevan</mark>, <mark>Franklin A.L. Chapman</mark>, <mark>Henry A. Kobulnicky</mark> |
|*Appeared on*| *2023-02-16*|
|*Comments*| *Submitted to AJ, 42 pages, 14 figures. arXiv admin note: substantial text overlap with arXiv:2201.09963*|
|**Abstract**| We confirm the planetary nature of two gas giants discovered by TESS to transit M dwarfs with stellar companions at wide separations. TOI-3984 A ($J=11.93$) is an M4 dwarf hosting a short-period ($4.353326 \pm 0.000005$ days) gas giant ($M_p=0.14\pm0.03~\mathrm{M_{J}}$ and $R_p=0.71\pm0.02~\mathrm{R_{J}}$) with a wide separation white dwarf companion. TOI-5293 A ($J=12.47$) is an M3 dwarf hosting a short-period ($2.930289 \pm 0.000004$ days) gas giant ($M_p=0.54\pm0.07~\mathrm{M_{J}}$ and $R_p=1.06\pm0.04~\mathrm{R_{J}}$) with a wide separation M dwarf companion. We characterize both systems using a combination of ground-based and space-based photometry, speckle imaging, and high-precision radial velocities from the Habitable-zone Planet Finder and NEID spectrographs. TOI-3984 A b ($T_{eq}=563\pm15$ K and $\mathrm{TSM}=138_{-27}^{+29}$) and TOI-5293 A b ($T_{eq}=675_{-30}^{+42}$ K and $\mathrm{TSM}=92\pm14$) are two of the coolest gas giants among the population of hot Jupiter-sized gas planets orbiting M dwarfs and are favorable targets for atmospheric characterization of temperate gas giants and three-dimensional obliquity measurements to probe system architecture and migration scenarios. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07761-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07761) | **The first JWST spectrum of a GRB afterglow: No bright supernova in  observations of the brightest GRB of all time, GRB 221009A**  |
|| A. J. Levan, et al. -- incl., <mark>E. Burns</mark>, <mark>C. L. Hedges</mark>, <mark>K. E. Heintz</mark> |
|*Appeared on*| *2023-02-16*|
|*Comments*| *Submitted to the Astrophysical Journal Letters for the GRB 221009A Special Issue. The results of this paper are under press embargo. 18 pages, 8 figures, 2 tables*|
|**Abstract**| We present JWST and Hubble Space Telescope (HST) observations of the afterglow of GRB\,221009A, the brightest gamma-ray burst (GRB) ever observed. Observations obtained with NIRSPEC (0.6-5.5 micron) and MIRI (5-12 micron) 12 days after the burst are the first mid-IR spectroscopy performed for a GRB. Assuming the underlying slope is that of a single power-law, we obtain $\beta \approx 0.35$ and $A_V = 4.9$, in excess of the notional Galactic value. This is suggestive of extinction above the notional Galactic value, possibly due to patchy extinction within the Milky Way or dust in the GRB host galaxy. It further implies that the X-ray and optical/IR regimes are not on the same branch of the synchrotron spectrum of the afterglow. If the cooling break lies between the X-ray and optical/IR, then the temporal declines would only match for a post jet break, ISM medium and electron index with $p<2$. The shape of the JWST spectrum is near-identical in the optical/nIR to X-shooter spectroscopy obtained at 0.5 days and to later time observations with HST. The lack of spectral evolution suggests the SNe is either substantially fainter or bluer than SN~1998bw. Our {\em HST} observations also reveal a disc-like host galaxy, viewed close to edge-on that further complicates the isolation of any supernova component. The host galaxy appears rather typical amongst long-GRB hosts and suggests that the extreme properties of GRB 221009A are not directly tied to its galaxy-scale environment. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07808-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07808) | **The discovery of blue-cored dwarf early-type galaxies in isolated  environments**  |
|| Soo-Chang Rey, Suk Kim, Jiwon Chung, <mark>Youngdae Lee</mark> |
|*Appeared on*| *2023-02-16*|
|*Comments*| *8 pages, 5 figures, accepted for publication in Astrophysical Journal*|
|**Abstract**| The presence of blue-cored dwarf early-type galaxies (dE(bc)s) in high-density environments supports the scenario of the transformation of infalling late-type galaxies into quiescent dwarf early-type galaxies by environmental effects. While low-density environments lacking environmental processes could not be relevant to the formation of dE(bc)s, we discovered a large sample of rare dE(bc)s in isolated environments at z < 0.01 using the NASA-Sloan Atlas catalog. Thirty-two isolated dE(bc)s were identified by visual inspection of the Sloan Digital Sky Survey images and g - r color profiles. We found that (1) isolated dE(bc)s exhibit similar structural parameters to dE(bc)s in the Virgo cluster; (2) based on the ultraviolet-r color-magnitude relation, color gradients, and optical emission lines of dE(bc)s, isolated dE(bc)s show more vigorous, centrally concentrated SF compared to their counterparts in the Virgo cluster; (3) at a given stellar mass, isolated dE(bc)s tend to have a larger fraction of gas mass than their Virgo counterparts. We discuss a scenario of episodic SF sustained by gas accretion, suggested by Sanchez Almeida et al., in which the star-bursting blue compact dwarf galaxy (BCD)-quiescent BCD (QBCD) cycle can be repeated during the Hubble time. We suggest that, in this cadence, isolated dE(bc)s might be QBCDs at pre- or post-BCD stages. Our results imply that dE(bc)s comprise a mixture of objects with two types of origins, nature or nurture, depending on their environment. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07853-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07853) | **The star formation history of the Sco-Cen association: Coherent star  formation patterns in space and time**  |
|| Sebastian Ratzenböck, et al. -- incl., <mark>Josefa E. Großschedl</mark> |
|*Appeared on*| *2023-02-16*|
|*Comments*| *19 pages, 14 figures, preliminary version of this work. Comments welcome. Soon to be submitted to A&A*|
|**Abstract**| We reconstruct the star formation history of the Sco-Cen OB association using a novel high-resolution age map of the region. We develop an approach to produce robust ages for Sco-Cen's recently identified 37 stellar clusters using the \texttt{SigMA} algorithm. The Sco-Cen star formation timeline reveals four periods of enhanced star formation activity, or bursts, remarkably separated by about 5 Myr. Of these, the second burst, which occurred 15 million years ago, is by far the dominant, and most of Sco-Cen's stars and clusters were in place by the end of this burst. The formation of stars and clusters in Sco-Cen is correlated, but not linearly, meaning that more stars were formed per cluster during the peak of star formation rate. Most of the clusters, which are large enough to have supernova precursors, were formed during the 15 Myr period. Star and cluster formation activity has been continuously declining since then. We have clear evidence that Sco-Cen formed from the inside out and contains 100-pc long correlated chains of contiguous clusters exhibiting well-defined age gradients, from massive older clusters to smaller young clusters. These observables suggest an important role for feedback in the formation of about half of Sco-Cen stars, although follow-up work is needed to quantify this statement. Finally, we confirm that the Upper-Sco age controversy discussed in the literature during the last decades is solved: the region toward Upper-Sco, a benchmark region for planet formation studies, contains not one but up to nine clusters spanning ages from 3 to 19 Myr. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2302.07277.md
    + _build/html/tmp_2302.07277/./figures/umap_raw_pg.png
    + _build/html/tmp_2302.07277/./figures/umap_images_pg_f200.png
    + _build/html/tmp_2302.07277/./figures/umap_images_pg_f356.png
    + _build/html/tmp_2302.07277/./figures/umap_classes_zoo.png
    + _build/html/tmp_2302.07277/./figures/umap_images_zoo_f200.png
    + _build/html/tmp_2302.07277/./figures/umap_images_zoo_f356.png
    + _build/html/tmp_2302.07277/./figures/umap_dist_umap_reduced_hexbin.png
    + _build/html/tmp_2302.07277/./figures/umap_CEERS_BACK356_hexbin.png
    + _build/html/tmp_2302.07277/./figures/umap_CEERS_log_rsersic_hexbin.png
    + _build/html/tmp_2302.07277/./figures/umap_CEERS_sersic_n_hexbin.png
    + _build/html/tmp_2302.07277/./figures/umap_CEERS_sersic_ellip_hexbin.png
    + _build/html/tmp_2302.07277/./figures/umap_CEERS_concentration_hexbin.png
    + _build/html/tmp_2302.07277/./figures/umap_CEERS_asymmetry_hexbin.png
    + _build/html/tmp_2302.07277/./

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\powerset}{\raisebox{.15\baselineskip}{\Large\ensuremath{\wp}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\powerset$}{\raisebox{.15\baselineskip}{\Large$\ensuremath${\wp}}}$</div>



<div id="title">

# On the nature of disks at high redshift seen by JWST/CEERS with contrastive learning and cosmological simulations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.07277-b31b1b.svg)](https://arxiv.org/abs/2302.07277)<mark>Appeared on: 2023-02-16</mark> - _25 pages, 23 figures. Submitted to ApJ. Comments welcome_

</div>
<div id="authors">

Jes\'us Vega-Ferrero, et al. -- incl., <mark>Annalisa Pillepich</mark>, <mark>Steven L. Finkelstein</mark>, <mark>Helena Domínguez Sánchez</mark>, <mark>Henry C. Ferguson</mark>

</div>
<div id="abstract">

**Abstract:** Visual inspections of the first optical rest-frame images from JWST have indicated a surprisingly high fraction of disk galaxies at high redshifts. Here we alternatively apply self-supervised machine learning to explore the morphological diversity at$z \geq 3$. Our proposed data-driven representation scheme of galaxy morphologies, calibrated on mock images from the TNG50 simulation, is shown to be robust to noise and to correlate well with physical properties of the simulated galaxies, including their 3D structure. We apply the method simultaneously to F200W and F356W galaxy images of a mass-complete sample ($M_*/M_\odot>10^9$) at$z \geq 3$from the first JWST/NIRCam CEERS data release. We find that the simulated and observed galaxies do not populate the same manifold in the representation space from contrastive learning, partly because the observed galaxies tend to be more compact and more elongated than the simulated galaxies. We also find that about half the galaxies that were visually classified as disks based on their elongated images actually populate a similar region of the representation space than spheroids, which according to the TNG50 simulation is occupied by objects with low stellar specific angular momentum and non-oblate structure. This suggests that the disk fraction at$z > 3$as evaluated by visual classification may be severely overestimated by misclassifying compact, elongated galaxies as disks. Deeper imaging and/or spectroscopic follow-ups as well as comparisons with other simulations will help to unambiguously determine the true nature of these galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2302.07277/./figures/umap_raw_pg.png" alt="Fig13.1" width="16%"/><img src="tmp_2302.07277/./figures/umap_images_pg_f200.png" alt="Fig13.2" width="16%"/><img src="tmp_2302.07277/./figures/umap_images_pg_f356.png" alt="Fig13.3" width="16%"/><img src="tmp_2302.07277/./figures/umap_classes_zoo.png" alt="Fig13.4" width="16%"/><img src="tmp_2302.07277/./figures/umap_images_zoo_f200.png" alt="Fig13.5" width="16%"/><img src="tmp_2302.07277/./figures/umap_images_zoo_f356.png" alt="Fig13.6" width="16%"/>

**Figure 13. -** Comparison of distributions of observed and simulated galaxies in the representation space. The top row shows the CEERS mass-complete sample and the bottom row the VISUAL sample with available visual morphologies. Top left-hand panel: UMAP visualization for the observed CEERS galaxy images selected in mass and redshift (color-coded by the source photo-$z$) overlapped with the representation of noise-added TNG50 galaxy images. Top middle panel: randomly chosen observed CEERS galaxy images in the UMAP visualization in the F200W filter. Top right-hand panel: randomly chosen observed CEERS galaxy images in the UMAP visualization in the F356W filter. Bottom left-hand panel: UMAP visualization for the observed VISUAL galaxy images selected in mass and redshift. Points are colored according to the visual classifications into several non-exclusive classes. Bottom middle panel: randomly chosen observed VISUAL galaxy images in the UMAP visualization in the F200W filter. Bottom right-hand panel: randomly chosen observed VISUAL galaxy images in the UMAP visualization in the F356W filter. (*fig:CEERS_umap*)

</div>
<div id="div_fig2">

<img src="tmp_2302.07277/./figures/umap_dist_umap_reduced_hexbin.png" alt="Fig9.1" width="50%"/><img src="tmp_2302.07277/./figures/umap_CEERS_BACK356_hexbin.png" alt="Fig9.2" width="50%"/>

**Figure 9. -** Left-hand panel: UMAP visualization for all the TNG50 galaxy images in our dataset color-coded by the mean value of the distance in the UMAP representation between the noiseless and the reduced images of each galaxy (denoted as $\delta$). Right-hand panel: UMAP visualization for all the TNG50 galaxy images in our dataset color-coded by the mean value of the ratio of the flux measured in the TNG50 stamps and the flux derived from the noiseless TNG50 stamps for the F356W filter (denoted as $\delta_\mathrm{F356W}$). Large values of $\delta_\mathrm{F356W}$ indicate the presence of a secondary (or even more) source. The larger $\delta_\mathrm{F356W}$ is, the brighter the companions are with respect to the central galaxy. (*fig:umap_CEERS_BACK356*)

</div>
<div id="div_fig3">

<img src="tmp_2302.07277/./figures/umap_CEERS_log_rsersic_hexbin.png" alt="Fig12.1" width="16%"/><img src="tmp_2302.07277/./figures/umap_CEERS_sersic_n_hexbin.png" alt="Fig12.2" width="16%"/><img src="tmp_2302.07277/./figures/umap_CEERS_sersic_ellip_hexbin.png" alt="Fig12.3" width="16%"/><img src="tmp_2302.07277/./figures/umap_CEERS_concentration_hexbin.png" alt="Fig12.4" width="16%"/><img src="tmp_2302.07277/./figures/umap_CEERS_asymmetry_hexbin.png" alt="Fig12.5" width="16%"/><img src="tmp_2302.07277/./figures/umap_CEERS_smoothness_hexbin.png" alt="Fig12.6" width="16%"/>

**Figure 12. -** UMAP visualization for all the TNG50 galaxy images in our dataset color-coded by the distribution of several morphological and photometric parameters. Color code corresponds to the median values in each hexagonal bin in the UMAP plane. From left to right and top to bottom, the different panels show: the logarithm of the effective radius ($r_e \mathrm{[kpc]}$), in kpc), the Sèrsic index ($n_e$), the ellipticity based on Sèrsic fit ($1-b/a$), the concentration ($C$), the asymmetry ($A$) and the smoothness ($C$). The scatter maps of these parameters are presented in \autoref{app:dependence}. (*fig:phot_hexbin_umap*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

#  OGLE-2017-BLG-1038: A Possible Brown-dwarf Binary Revealed by Spitzer Microlensing Parallax

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.07497-b31b1b.svg)](https://arxiv.org/abs/2302.07497)<mark>Appeared on: 2023-02-16</mark> - _20 pages, 11 figures, 4 tables_

</div>
<div id="authors">

Amber Malpas, et al. -- incl., <mark><mark>Andrew Gould</mark></mark>, <mark><mark>Antonio Herrera Martin</mark></mark>, <mark><mark>Calen B. Henderson</mark></mark>, <mark><mark>Seung-Lee Kim</mark></mark>, <mark><mark>Chung-Uk Lee</mark></mark>, <mark><mark>Dong-Joo Lee</mark></mark>, <mark><mark>Yongseok Lee</mark></mark>

</div>
<div id="abstract">

**Abstract:** We report the analysis of microlensing event OGLE-2017-BLG-1038, observed by the Optical Gravitational Lensing Experiment, Korean Microlensing Telescope Network, and Spitzer telescopes. The event is caused by a giant source star in the Galactic Bulge passing over a large resonant binary lens caustic. The availability of space-based data allows the full set of physical parameters to be calculated.However, there exists an eightfold degeneracy in the parallax measurement. The four best solutions correspond to very-low-mass binaries near ($M_1 = 170^{+40}_{-50} M_J$and$M_2 = 110^{+20}_{-30} M_J$), or well below ($M_1 = 22.5^{+0.7}_{-0.4} M_J$and$M_2 = 13.3^{+0.4}_{-0.3} M_J$) the boundary between stars and brown dwarfs.A conventional analysis, with scaled uncertainties for Spitzer data, implies a very-low-mass brown dwarf binary lens at a distance of 2 kpc. Compensating for systematic Spitzer errors using a Gaussian process model suggests that a higher mass M-dwarf binary at 6 kpc is equally likely.A Bayesian comparison based on a galactic model favors the larger-mass solutions.We demonstrate how this degeneracy can be resolved within the next ten years through infrared adaptive-optics imaging with a 40 m class telescope.

</div>

<div id="div_fig1">

<img src="tmp_2302.07497/./spitzer_grid-paper-annotated.png" alt="Fig4.1" width="50%"/><img src="tmp_2302.07497/./Jen_Caustic_combined_spitzer-constrained_renorm-annotated.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** \emph{Left}: contour maps demonstrating the results of the parallax grid searches over discretely varied $\pi_{{\rm E},E}$ and $\pi_{{\rm E},N}$ for the $-u_{0,Earth}$ configuration, including only the Spitzer $\chi^2$ components. The dashed contours show the $\chi^2$ landscape without a color constraint and the solid lines with the constraint. Note that the $\pi_{{\rm E},N}$-axis  of this figure is reversed from the usual orientation so that the two figures approximately align. \emph{Right}: caustic diagram with projected ground-based and Spitzer-based trajectories (black and red, respectively).  The four Spitzer trajectories are the result of minimization from the local $\chi^{2}$ minima from the left figure, with all modeling parameters free to evolve.  The data points are represented by colored circles on the trajectories, where the colors correspond to the observation site and field, as specified in Figures \ref{Fig:StaticLC} and \ref{Fig:GroundCumulative}. The caustics change with the orbiting of the lens bodies and are depicted here at the instances of the first and last Spitzer data points, specifically for the c -/+ solution (all four $u_{0,Earth}<0$ solutions looks very diagrammatically similar to the one shown). These epochs are represented on grounds-based trajectory (also from the c -/+ solution) with colours matching their corresponding caustics. (*Fig:RSGrid*)

</div>
<div id="div_fig2">

<img src="tmp_2302.07497/./pm_theoretical_disk.png" alt="Fig3" width="100%"/>

**Figure 3. -** Proper motions of the lens solutions and source star. For context, we also include contour representations of the disk (blue) and bulge (red) distributions. The bulge contours are from histograms of the red-clump stars from Gaia EDR3, selecting stars within a $0.2^o$ radius cone centred on the lens. The distribution of red-clump stars is the results of a Gaussian fit to the red clump on the field's CMD. The innermost thicker line of the red-clump distribution contains approximately 68\% of the population samples. The outermost thicker line contains approximately  95\% of the population samples. The blue contours depict the theoretical distribution of the disk stars used in our galactic model. The solid ellipses correspond to the 1 and 2 $\sigma$ proper-motion dispersions of disk stars at $D=6$ kpc. The dotted ellipses show the same for disk stars at $D=2.3$ kpc.
 (*F:pm*)

</div>
<div id="div_fig3">

<img src="tmp_2302.07497/./naive_Spitzer_lightcurve_w_res.png" alt="Fig1" width="100%"/>

**Figure 1. -** Raw Spitzer light curve and model light curve resulting from the static binary-lens model (no parallax), fitted to the ground-based data, and transforming to the Spitzer flux system assuming $F_{\rm L} \equiv F_{{\rm S},L} A$, where $F_{{\rm S},L} = 56.1$ and $(I - L)_{\rm S} = -7.4$ . The ground-based observations have also been scaled to the Spitzer flux system. The residuals between the model and the data are depicted in black for ground-based data and red for \textit{Spitzer} data. These show a dramatic difference for the $t<7935$ data. \label{Fig:SpitzerLC} (*Fig:SpitzerLC*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

206  publications in the last 7 days.
	 _build/html/2302.07497.md
	 _build/html/2302.07277.md
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/html/2301.08770.md
	 _build/html/2301.08310.md
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers